# Apple Leaf Disease Recognition Using Deep Learning



*   [Baseline Model](https://www.google.com/url?q=https%3A%2F%2Fwww.kaggle.com%2Fcode%2Fchanchal24%2Fplant-disease-recognition-using-dl%2Fnotebook)
*   [Dataset](https://www.google.com/url?q=https%3A%2F%2Fwww.kaggle.com%2Fdatasets%2Frashikrahmanpritom%2Fplant-disease-recognition-dataset)



In [ ]:
pip install torch torchvision tensorflow keras numpy matplotlib seaborn

# Loading the Dataset

In [ ]:
# PyTorch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# TensorFlow/Keras Libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    BatchNormalization, ReLU, GlobalAveragePooling2D, ZeroPadding2D
)
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


# General Utilities
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import shutil

# Visualization
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Function to count the total files in a directory
def total_files(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

In [ ]:
# Root directories for training, validation, and testing
#Charbel's path
train_dir = "/content/drive/MyDrive/AML_Dataset/Train/Train"
val_dir = "/content/drive/MyDrive/AML_Dataset/Validation/Validation"
test_dir = "/content/drive/MyDrive/AML_Dataset/Test/Test"

#Nakhla's path
# train_dir="/content/drive/MyDrive/AML_Dataset/Train/Train"
# val_dir = "/content/drive/MyDrive/AML_Dataset/Validation/Validation"
# test_dir = "/content/drive/MyDrive/AML_Dataset/Test/Test"
# Target image size and batch size
image_size = (225, 225)
batch_size = 32

# Class-specific paths
train_files_healthy = f"{train_dir}/Healthy"
train_files_powdery = f"{train_dir}/Powdery"
train_files_rust = f"{train_dir}/Rust"

test_files_healthy = f"{test_dir}/Healthy"
test_files_powdery = f"{test_dir}/Powdery"
test_files_rust = f"{test_dir}/Rust"

valid_files_healthy = f"{val_dir}/Healthy"
valid_files_powdery = f"{val_dir}/Powdery"
valid_files_rust = f"{val_dir}/Rust"

# Print file counts for each class
print("Number of healthy leaf images in training set", total_files(train_files_healthy))
print("Number of powder leaf images in training set", total_files(train_files_powdery))
print("Number of rusty leaf images in training set", total_files(train_files_rust))

print("========================================================")

print("Number of healthy leaf images in test set", total_files(test_files_healthy))
print("Number of powder leaf images in test set", total_files(test_files_powdery))
print("Number of rusty leaf images in test set", total_files(test_files_rust))

print("========================================================")

print("Number of healthy leaf images in validation set", total_files(valid_files_healthy))
print("Number of powder leaf images in validation set", total_files(valid_files_powdery))
print("Number of rusty leaf images in validation set", total_files(valid_files_rust))

# Visualizing Images for Different Classes

In [ ]:
# Helper function to display an image from a given directory and filename
def display_image(directory, filename, width=500):
    image_path = os.path.join(directory, filename)
    with open(image_path, 'rb') as f:
        display.display(display.Image(data=f.read(), width=width))

## Healthy

In [ ]:
print("Displaying a sample image from the Healthy class (Train):")
display_image(train_files_healthy, '8ce77048e12f3dd4.jpg')

## Rust

In [ ]:
print("Displaying a sample image from the Rust class (Train):")
display_image(train_files_rust, '80f09587dfc7988e.jpg')

## Powdery

In [ ]:
print("Displaying a sample image from the Powdery class (Train):")
display_image(train_files_powdery, '8b7569be32192c1e.jpg')

# Data Augmentation

In [ ]:
augmented_train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'        # Fill pixels after transformations
)

In [ ]:
# Now, we will create the directory to save the train augmented data
# Directory to store augmented images
#Charbel's Path
# augmented_train_dir = "/content/drive/MyDrive/Advanced ML/Final Project/AML_Dataset/Augmented_Train"

#Nakhla's path
augmented_train_dir= "/content/drive/MyDrive/AML_Dataset/Augmented_Train"

# Create directories for each class
for class_dir in ['Healthy', 'Powdery', 'Rust']:
    class_path = os.path.join(augmented_train_dir, class_dir)
    if os.path.exists(class_path):
        shutil.rmtree(class_path)  # Clear existing files if directory exists
    os.makedirs(class_path)  # Create fresh directory

In [ ]:
# Augmented data generator
augmented_generator = augmented_train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=1,  # Save one image at a time
    class_mode='categorical',
    shuffle=True  # Shuffle to get varied batches
)

# Generate and save augmented images in class-specific folders
num_augmented_images = 1000  # Total number of augmented images to create
class_indices = augmented_generator.class_indices  # Mapping of classes to indices
reverse_class_indices = {v: k for k, v in class_indices.items()}  # Reverse mapping

for i in range(num_augmented_images):
    batch = next(augmented_generator)  # Get the next batch (image, label)
    image, label = batch[0], batch[1]

    # Find the class label from the one-hot encoded label
    class_index = label.argmax()  # Get index of the class
    class_name = reverse_class_indices[class_index]  # Get the class name

    # Save the image in the corresponding class folder
    save_dir = os.path.join(augmented_train_dir, class_name)
    augmented_generator.save_to_dir = save_dir  # Dynamically update save directory


In [ ]:
# Function to count files in a directory
def total_files(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

print("Updated counts for augmented training dataset:")
for class_dir in ['Healthy', 'Powdery', 'Rust']:
    class_path = os.path.join(augmented_train_dir, class_dir)
    print(f"Number of images in {class_dir} class: {total_files(class_path)}")


## Adding the Augmented train data, to the original one

In [ ]:

# Create the combined directory
if os.path.exists(combined_train_dir):
    shutil.rmtree(combined_train_dir)  # Clear existing directory
os.makedirs(combined_train_dir)

# Copy images from both original and augmented datasets into combined directory
for class_name in ['Healthy', 'Powdery', 'Rust']:
    # Create class subdirectories in the combined directory
    class_combined_dir = os.path.join(combined_train_dir, class_name)
    os.makedirs(class_combined_dir)

    # Copy images from the original dataset
    original_class_dir = os.path.join(original_train_dir, class_name)
    for file_name in os.listdir(original_class_dir):
        src_path = os.path.join(original_class_dir, file_name)
        dst_path = os.path.join(class_combined_dir, file_name)
        shutil.copy(src_path, dst_path)

    # Copy images from the augmented dataset
    augmented_class_dir = os.path.join(augmented_train_dir, class_name)
    for file_name in os.listdir(augmented_class_dir):
        src_path = os.path.join(augmented_class_dir, file_name)
        dst_path = os.path.join(class_combined_dir, file_name)
        shutil.copy(src_path, dst_path)

print("Combined dataset created successfully!")

In [ ]:
# Function to count total files in a directory
def total_files(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

# Print counts for each class in the combined dataset
print("Image counts in Combined_Train:")
for class_name in ['Healthy', 'Powdery', 'Rust']:
    class_path = os.path.join(combined_train_dir, class_name)
    print(f"{class_name}: {total_files(class_path)} images")

Data Successfully augmented!

# Preprocessing For Pytorch

Checking for cuda and loading to gpu

In [ ]:
# Check GPU availability
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU found.")

# Assign device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Preprocessing images

In [ ]:
trans=transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
])

#Original Dataset
train=datasets.ImageFolder("/content/drive/MyDrive/AML_Dataset/Train/Train",transform=trans)
validation=datasets.ImageFolder("/content/drive/MyDrive/AML_Dataset/Validation/Validation",transform=trans)
test=datasets.ImageFolder("/content/drive/MyDrive/AML_Dataset/Test/Test",transform=trans)

#Augmented Dataset

class_names=train.classes

all_data = [data for data in train]
validationData=[data for data in validation]
testData=[data for data in test]


Pytorch implementation for dataloaders

In [ ]:
batch=64
train_loader = DataLoader(train, batch_size=batch, num_workers=4, pin_memory=True, shuffle=True)
validation_loader = DataLoader(validation, batch_size=batch, num_workers=4, pin_memory=True)
test_loader = DataLoader(test, batch_size=batch, num_workers=4, pin_memory=True)

# Preprocessing For TensorFlow

In [ ]:
# Directories
# #Charbel's path
original_train_dir = "/content/drive/MyDrive/Advanced ML/Final Project/AML_Dataset/Train/Train"
augmented_train_dir = "/content/drive/MyDrive/Advanced ML/Final Project/AML_Dataset/Augmented_Train"
combined_train_dir = "/content/drive/MyDrive/Advanced ML/Final Project/AML_Dataset/Combined_Train"

#Nakhla's path
# original_train_dir = "/content/drive/MyDrive/AML_Dataset/Train/Train"
# augmented_train_dir = "/content/drive/MyDrive/AML_Dataset/Augmented_Train"
# combined_train_dir = "/content/drive/MyDrive/AML_Dataset/Combined_Train"

Unaugmented

In [ ]:
# Preprocessing for the unaugmented data
train_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Combined

In [ ]:
# Preprocessing for combined dataset
train_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    combined_train_dir,
    # target_size=image_size,
    target_size=(256, 256) ,
    batch_size=batch_size,
    class_mode='categorical'
)

Validation and testing

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Creating generators
validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    # target_size=image_size,
    target_size=(256, 256) ,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    # target_size=image_size,
    target_size=(256, 256) ,
    batch_size=batch_size,
    class_mode='categorical'
)

Inception

In [ ]:
# Define image size and batch size
img_size = (256, 256)
batch_size = 32

# Data preprocessing and augmentation using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators for loading the data
train_generator = train_datagen.flow_from_directory(
    combined_train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Baseline Model

In [ ]:
model_baseline = Sequential()
# Input layer
model_baseline.add(Input(shape=(225, 225, 3)))

# First Conv Block
model_baseline.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model_baseline.add(MaxPooling2D(pool_size=(2, 2)))

# Second Conv Block
model_baseline.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
model_baseline.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and Dense Layers
model_baseline.add(Flatten())
model_baseline.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))

# Output Layer
model_baseline.add(Dense(3, activation='softmax'))

# Updated Baseline Model

In [ ]:
model_baseline2 = Sequential()

# First Conv Block
model_baseline2.add(Conv2D(32, (3, 3), input_shape=(225, 225, 3), activation='relu'))
model_baseline2.add(MaxPooling2D(pool_size=(2, 2)))

# Second Conv Block
model_baseline2.add(Conv2D(64, (3, 3), activation='relu'))
model_baseline2.add(MaxPooling2D(pool_size=(2, 2)))

# Third Conv Block
model_baseline2.add(Conv2D(128, (3, 3), activation='relu'))
model_baseline2.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth Conv Block
model_baseline2.add(Conv2D(128, (3, 3), activation='relu'))
model_baseline2.add(MaxPooling2D(pool_size=(2, 2)))

# Fifth Conv Block
model_baseline2.add(Conv2D(256, (3, 3), activation='relu'))
model_baseline2.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten Layer
model_baseline2.add(Flatten())

# Fully Connected Layers
model_baseline2.add(Dense(256, activation='relu'))
model_baseline2.add(Dense(128, activation='relu'))

# Output Layer
model_baseline2.add(Dense(3, activation='softmax'))  # 3 classes: Healthy, Powdery, Rust

# CNN 1

In [ ]:
model_cnn = Sequential()

# First Conv Block
model_cnn.add(Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(225, 225, 3)))
model_cnn.add(Conv2D(32, (3, 3), activation="relu", padding="same"))
model_cnn.add(MaxPooling2D(pool_size=(3, 3)))

# Second Conv Block
model_cnn.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model_cnn.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
model_cnn.add(MaxPooling2D(pool_size=(3, 3)))

# Third Conv Block
model_cnn.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model_cnn.add(Conv2D(128, (3, 3), activation="relu", padding="same"))
model_cnn.add(MaxPooling2D(pool_size=(3, 3)))

# Fourth Conv Block
model_cnn.add(Conv2D(256, (3, 3), activation="relu", padding="same"))
model_cnn.add(Conv2D(256, (3, 3), activation="relu", padding="same"))

# Fifth Conv Block
model_cnn.add(Conv2D(512, (5, 5), activation="relu", padding="same"))
model_cnn.add(Conv2D(512, (5, 5), activation="relu", padding="same"))

# Flatten and Dense Layers
model_cnn.add(Flatten())
model_cnn.add(Dense(1568, activation="relu"))
model_cnn.add(Dropout(0.5))  # Added dropout for regularization
model_cnn.add(Dense(3, activation="softmax"))

# Pytorch CNN

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet,self).__init__()
    self.network=nn.Sequential(
        nn.ZeroPad2d(3),
        nn.Conv2d(in_channels=3,out_channels=16,kernel_size=5,stride=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=2),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(576,3),
    )
  def forward(self,x):
    x=self.network(x)
    return x

In [ ]:
model_cnn2=NeuralNet().to(device)

In [ ]:
pip install torchviz


# ResNet 34

In [ ]:
# Define the residual block for ResNet
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1): # Changed _init_ to __init__
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out

# Define the ResNet-34 model
class ResNet34(nn.Module):
    def __init__(self, num_classes=38): # Changed _init_ to __init__
        super(ResNet34, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Define the stages
        self.layer1 = self._make_layer(64, 64, 3)
        self.layer2 = self._make_layer(64, 128, 4, stride=2)
        self.layer3 = self._make_layer(128, 256, 6, stride=2)
        self.layer4 = self._make_layer(256, 512, 3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [ ]:
# Instantiate the model
model = ResNet34(num_classes=len(class_names))

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Load data
train_loader = DataLoader(train, batch_size=64, shuffle=True, num_workers=4)
validation_loader = DataLoader(validation, batch_size=64, shuffle=False, num_workers=4)

# Initialize lists to store loss and accuracy values
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Training loop
epochs = 7
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in validation_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = running_val_loss / len(validation_loader)
    val_accuracy = 100 * correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

print("Training complete.")

In [ ]:
# Plotting loss and accuracy curves
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy Curves')
plt.legend()

plt.tight_layout()
plt.show()

# RNN

In [ ]:
model_rnn = Sequential()

# Reshape the input to (time steps, features) for RNN layers
model_rnn.add(Reshape((225, 225*3), input_shape=(225, 225, 3)))  # Adjusted to match (225, 225, 3)

# Stacked SimpleRNN layers
model_rnn.add(SimpleRNN(128, return_sequences=True))
model_rnn.add(SimpleRNN(128, return_sequences=True))
model_rnn.add(SimpleRNN(128))

# Fully Connected Layers
model_rnn.add(Dense(1568, activation="relu"))
model_rnn.add(Dropout(0.5))  # Regularization to prevent overfitting

# Output Layer
model_rnn.add(Dense(3, activation="softmax"))

# LSTM

In [ ]:
model_lstm = Sequential()

# Reshape the input to (time steps, features) for LSTM layers
model_lstm.add(Reshape((225, 225*3), input_shape=(225, 225, 3)))  # Adjusted to match (225, 225, 3)

# Add LSTM layers
model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(LSTM(128))

# Fully Connected Layers
model_lstm.add(Dense(1568, activation="relu"))
model_lstm.add(Dropout(0.5))  # Regularization to prevent overfitting

# Output Layer
model_lstm.add(Dense(3, activation="softmax"))

# Inception

In [ ]:
# Load the InceptionV3 model pre-trained on ImageNet without the top layer (for transfer learning)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

base_model.trainable = True
for layer in base_model.layers[:-30]:  # Freeze all but the top 30 layers
    layer.trainable = False

# Custom classifier with more capacity
model_inception = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),  # Additional dense layer
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])


# Tensorflow Modeling

In [ ]:
# default_loss = 'categorical_crossentropy'
default_loss = 'categorical_crossentropy'
default_metrics = ['accuracy']
default_learning_rate = 1e-5

def compile_model(model, loss=default_loss, metrics=default_metrics, learning_rate=default_learning_rate):
    optimizer_instance = Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer_instance, loss=loss, metrics=metrics)
    print(f"Model compiled with Adam optimizer (learning_rate={learning_rate}), loss={loss}, metrics={metrics}")
    model.summary()


In [ ]:
def train_model(model, train_data, validation_data, epochs=5, batch_size=16):
    print(f"Starting training for {epochs} epochs with batch size {batch_size}...")
    history = model.fit(
        train_data,
        epochs=epochs,
        validation_data=validation_data,
        batch_size=batch_size
    )
    print("Training complete!")
    return history

In [ ]:
compile_model(model_inception)

In [ ]:
# # # Train the model
# history = train_model(
#     model_inception,
#     train_data=train_generator,
#     validation_data=validation_generator,
#     epochs=15,
#     batch_size=16,
# )

#Inception
# Train the model
epochs = 10
history = model_inception.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)


In [ ]:
model_inception.save("inception_model.h5")

#Pytorch Modelling & Plotting

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_cnn2.parameters(), lr=0.0003)

In [ ]:
def train(train_loader):
    model_cnn2.train()
    loss_per_epoch = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()  # Clear previous gradients
        outputs = model_cnn2(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        loss_per_epoch += loss.item()
        preds = torch.argmax(outputs, dim=1)  # Get predicted labels
        correct += (preds == labels).sum().item()  # Count correct predictions
        total += labels.size(0)  # Update total count

    train_accuracy = correct / total  # Compute accuracy
    return loss_per_epoch, train_accuracy


In [ ]:
def validate(val_loader):
    model_cnn2.eval()
    loss_per_epoch = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_cnn2(images)
            loss = criterion(outputs, labels)

            loss_per_epoch += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_accuracy = correct / total  # Compute accuracy
    return loss_per_epoch, val_accuracy


In [ ]:
epochs=10
train_loss = []
val_loss = []
train_acc = []
val_acc = []

for i in range(epochs):
    # Training
    loss_train, acc_train = train(train_loader)
    # Validation
    loss_val, acc_val = validate(validation_loader)

    # Store results
    train_loss.append(loss_train)
    val_loss.append(loss_val)
    train_acc.append(acc_train)
    val_acc.append(acc_val)

    print(f"Epoch {i+1}/{epochs} -> Train Loss: {loss_train:.4f}, Train Acc: {acc_train:.4f}, "
          f"Val Loss: {loss_val:.4f}, Val Acc: {acc_val:.4f}")
plt.plot(train_loss,label="training loss")
plt.plot(val_loss,label="validation loss")
plt.legend()
plt.show()

In [ ]:
torch.save(model_cnn2.state_dict(), "model_cnn.pth")
print("Model weights saved successfully.")

In [ ]:
# def test(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     model.eval()
#     test_loss, correct = 0, 0

#     with torch.no_grad():
#         for X, y in dataloader:
#             X, y = X.float().to(device), y.to(device)
#             pred = model(X)
#             test_loss += loss_fn(pred, y).item()
#             preds = torch.argmax(pred, dim=1)
#             correct += (preds == y).sum().item()

#     test_loss /= num_batches
#     test_accuracy = correct / size  # Compute accuracy
#     print(f"Test Error: \n Accuracy: {(100*test_accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")
#     return test_loss, test_accuracy

# test(test_loader,model_cnn2,criterion)

# Tensorflow Plotting Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set_context("poster")

def plot_accuracy(history, figsize=(10, 6), dpi=100):
    # Use plt.figure instead of just figure
    plt.figure(figsize=figsize, dpi=dpi)
    plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
    plt.title('Model Accuracy', fontsize=16)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.grid(True)
    plt.show()

def plot_loss(history, figsize=(10, 6), dpi=100):
    # Use plt.figure instead of just figure
    plt.figure(figsize=figsize, dpi=dpi)
    plt.plot(history.history['loss'], label='Train Loss', marker='o')
    plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
    plt.title('Model Loss', fontsize=16)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

In [ ]:
# Plot accuracy
plot_accuracy(history, figsize=(12, 8), dpi=100)

# Plot loss
plot_loss(history, figsize=(12, 8), dpi=100)

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history["train_loss"], label="Training Loss")
    plt.plot(history["val_loss"], label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss Over Epochs")
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history["train_acc"], label="Training Accuracy")
    plt.plot(history["val_acc"], label="Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy Over Epochs")
    plt.legend()

    plt.tight_layout()
    plt.show()


# Model Deployment

In [ ]:
# model_cnn= load_model('model_cnn.h5')

In [ ]:
def preprocess_image(image_path, target_size=(256, 256)):  # Change target size to (256, 256)
    """
    Preprocesses an image for prediction.
    """
    try:
        # Load the image with the target size
        img = load_img(image_path, target_size=target_size)  # Resize to (256, 256)
        # Convert image to numpy array
        x = img_to_array(img)
        # Normalize pixel values to [0, 1]
        x = x.astype('float32') / 255.0
        # Add a batch dimension (required for prediction)
        x = np.expand_dims(x, axis=0)
        return x
    except Exception as e:
        print(f"Error preprocessing image at {image_path}: {e}")
        return None


In [ ]:
# Usage
# image_path = '/content/drive/MyDrive/Advanced ML/Final Project/AML_Dataset/Test/Test/Rust/82f49a4a7b9585f1.jpg'

#Nakhla's path
image_path = '/content/drive/MyDrive/AML_Dataset/Test/Test/Rust/82f49a4a7b9585f1.jpg'

# Test the function again
x = preprocess_image(image_path)

if x is not None:
    print("Image preprocessed successfully!")
    print(f"Shape of preprocessed image: {x.shape}")
else:
    print("Failed to preprocess image.")

In [ ]:
# Predict class probabilities for the preprocessed image
predictions = model_cnn2.predict(x)

# Display raw predictions (probabilities)
print("Raw predictions (class probabilities):", predictions[0])

In [ ]:
# Retrieve and reverse class indices
labels = train_generator.class_indices
labels = {v: k for k, v in labels.items()}
print("Class labels mapping:", labels)

In [ ]:
# Find the predicted class
predicted_index = np.argmax(predictions[0])
predicted_label = labels[predicted_index]

# Display the predicted label
print(f"Predicted Label: {predicted_label}")

# Demo

In [ ]:
import torch
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor, Resize, Normalize, Compose
from PIL import Image

# Update preprocessing for PyTorch
def preprocess_image(image_path, target_size=(256, 256)):
    original_img = Image.open(image_path).convert("RGB")  # Load and convert to RGB
    transform = Compose([
        Resize(target_size),  # Resize the image
        ToTensor(),           # Convert to tensor
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize (ImageNet stats)
    ])
    preprocessed_img = transform(original_img).unsqueeze(0)  # Add batch dimension
    return preprocessed_img, original_img

def predict_and_display(image_path, model, class_names):
    # Preprocess the image
    preprocessed_img, original_img = preprocess_image(image_path)

    # Move the image and model to the device
    preprocessed_img = preprocessed_img.to(device)
    model = model.to(device)
    model.eval()  # Set the model to evaluation mode

    # Perform inference
    with torch.no_grad():
        outputs = model(preprocessed_img)  # Forward pass
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)  # Softmax for probabilities
        predicted_class_index = torch.argmax(probabilities).item()
        predicted_class_name = class_names[predicted_class_index]
        confidence_score = probabilities[predicted_class_index].item()

    # Display the image, predicted class, and confidence
    plt.figure(figsize=(6, 6))
    plt.imshow(original_img)
    plt.axis("off")
    plt.title(f"Predicted: {predicted_class_name}\nConfidence: {confidence_score:.2f}", fontsize=12)
    plt.show()

    # Print confidence scores for each class
    print("Confidence scores for each class:")
    for i, score in enumerate(probabilities):
        print(f"{class_names[i]}: {score:.2f}")

In [ ]:
class_names = ["Healthy", "Powdery", "Rust"]  # Update with your actual class names

Healthy

In [ ]:
# Example usage with an image
image_path = "/content/drive/MyDrive/AML_Dataset/Test/Test/Healthy/8ddd5ec1c0de38c4.jpg"
predict_and_display(image_path, model_cnn2, class_names)

Rust

In [ ]:
# Example usage with an image
image_path = "/content/drive/MyDrive/AML_Dataset/Test/Test/Rust/90bb8dcc6f0c58b9.jpg"
predict_and_display(image_path, model_cnn2, class_names)